<a href="https://colab.research.google.com/github/caiocezar400/DeepSea-Solutions/blob/main/Cinematica_inversa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sympy as sp
import numpy as np
from sympy import symbols, cos, sin, pi, simplify, Matrix, pprint, sympify

def dh_matrix(a, alpha, d, theta):
    """
    DH_MATRIX Denavit-Hartenberg
    Calcula a matriz de transformação homogênea 4x4
    """
    T = sp.Matrix([
        [cos(theta), -sin(theta)*cos(alpha),  sin(theta)*sin(alpha), a*cos(theta)],
        [sin(theta),  cos(theta)*cos(alpha), -cos(theta)*sin(alpha), a*sin(theta)],
        [0,           sin(alpha),             cos(alpha),            d],
        [0,           0,                      0,                     1]
    ])
    return T

def configurar_robo():
  # Solicita a configuração do robô (ex: RRR, RRP, RRRP)
  config_str = input('\nConfiguração do robô (ex: RRR, RRP, RP): ').strip().upper()

  # Remove caracteres inválidos se houver, mantendo apenas R ou P
  joints = [j for j in config_str if j in ['R', 'P']]
  N = len(joints)

  if N == 0:
    print("Erro: Nenhuma junta válida encontrada. Use 'R' para Revoluta ou 'P' para Prismática.")
    return

  print(f'\nRobô detectado com {N} juntas: {"-".join(joints)}')
  print('Para cada junta, insira os parâmetros constantes.')
  print('A variável da junta (q_i) será adicionada automaticamente.')
  print('(Pode usar valores simbólicos como L1, pi/2, d3, etc.)')

  dh_params = []

  for i, junta_tipo in enumerate(joints):
    idx = i + 1 # Índice da junta (1-based)

    # Cria a variável simbólica automática para esta junta (q1, q2, etc.)
    q_var = symbols(f'q{idx}')

    # Descrição do tipo de junta para o usuário
    tipo_desc = "Revoluta (R)" if junta_tipo == 'R' else "Prismática (P)"
    print(f'\n--- Junta {idx}: {tipo_desc} ---')

    # Inputs comuns a ambos os tipos
    a_str = input('a (cm): ').strip() or '0'
    alpha_str = input('alpha (rad): ').strip() or '0'

    # Converte para simbólico
    a = sympify(a_str)
    alpha = sympify(alpha_str)

    if junta_tipo == 'R':
      # Se é Revoluta: 'd' é constante, 'theta' é variável (q_i + offset)
      d_str = input('d (cm): ').strip() or '0'
      theta_offset_str = input('theta offset (rad) [Enter para 0]: ').strip() or '0'

      d = sympify(d_str)
      theta_offset = sympify(theta_offset_str)

      # Define theta como a variável q + o offset constante
      theta = q_var + theta_offset

      print(f'-> Configurado: theta = {theta}')

    elif junta_tipo == 'P':
      # Se é Prismática: 'theta' é constante, 'd' é variável (q_i + offset)
      theta_str = input('theta (rad): ').strip() or '0'
      d_offset_str = input('d offset (cm) [Enter para 0]: ').strip() or '0'

      theta = sympify(theta_str)
      d_offset = sympify(d_offset_str)

      # Define d como a variável q + o offset constante
      d = q_var + d_offset

      print(f'-> Configurado: d = {d}')

    # Converte entradas string em valores simbólicos
    params = [a, alpha, d, theta]
    dh_params.append(params)

  return dh_params, joints

def calculate_inverse_kinematic():
    """
    Calcula os passos intermediários de T0_3 a T2_3.
    """

    print("\n###### Cálculo de cinemática inversa ########")

    dh_params, _ = configurar_robo()
    N = len(dh_params)

    # Cria uma lista de todas as matrizes de transformação
    A = [dh_matrix(*params) for params in dh_params]

    # Calcula a matriz de transformação final T0_3
    # T0_3 = A0_1 * A1_2 * A2_3
    T03 = sp.Identity(4)
    for matrix in A:
        T03 *= matrix

    # As interações começam a partir da T0_(num_joints)
    T_current = T03

    print("\n###### Preenchimento da matriz conhecida ########")

    # Cria uma matriz 4x4 preenchida com zeros
    Mc = np.zeros((4, 4))

    labels = [
        ['nx', 'sx', 'ax', 'px'],
        ['ny', 'sy', 'ay', 'py'],
        ['nz', 'sz', 'az', 'pz']
    ]

    # Carrega os valores conhecidos em uma matriz Mc
    for i in range(3):
        for j in range(4):
            element_name = labels[i][j]

            value = float(input(f"\nDefina o elemento {element_name}: "))
            Mc[i, j] = value

    # Finaliza a matriz, sendo a última linha padronizada
    Mc[3, :] = [0, 0, 0, 1]

    # Mostra a matriz montada para confirmação
    print("\nA matriz registrada é a seguinte:")
    sp.pprint(Mc)

    ans = input("\nConfirma? (y/n) ")

    if (ans.lower() != "y"):
      return

    # Converte numérico em simbólico
    Mc_sym = Matrix(Mc)

    # Inicializa A_inv_left
    A_inv_left = sp.Identity(4)

    # Iterativamente aplica-se as inversões de martrizes
    for i in range(N):
        # Mc = matriz conhecida, de forma 4x4: [nx  sx  ax  px
        #                                      ny  sy  ay  py
        #                                      nz  sz  az  pz
        #                                      0   0   0   1]

        # 1. A0_1_inv * Mc = T1_3 = A0_1_inv * T0_3
        # 2. A1_2_inv * A0_1_inv * Mc = T2_3 = A1_2_inv * T1_3
        # ...

        ###### Lado esquerdo do equacionamento (conhecido)

        # Inverte a matriz A de posição i
        A_inv_left *= A[i].inv()

        # Aplicando as multiplicações
        Tc = A_inv_left * Mc_sym

        # Simplificando para facilidade de leitura
        Tc = simplify(Tc)

        ###################################################

        ###### Lado direito do equacionamento (não conhecido)

        # Inverte a matriz A de posição i
        A_inv_right = A[i].inv()

        # Aplicando as multiplicações
        T_next = A_inv_right * T_current

        # Simplificando o resultado para facilidade de leitura
        T_next = simplify(T_next)

        # Atualiza a matriz para a próxima iteração
        T_current = T_next

        #####################################################

        # Demonstra o resultado de cada iteração das matrizes conhecidas
        print("\n")
        print("=" * 45)
        print(f"Resultado da Matriz conhecida da {i+1}° iteração")
        print("=" * 45)
        sp.pprint(Tc)
        print("\n")

        # Demonstra o resultado de cada iteração
        print("\n")
        print("=" * 25)
        print(f"Resultado da Matriz T{i+1}_3")
        print("=" * 25)
        sp.pprint(T_next)
        print("\n")

    # Cálculo numérico, se possível
    # evaluate_matrices(Tc, T_next)

def evaluate_matrices(Tc, T_next):
        """
        Soluciona o sistema de matrizes produzido na
        calculate_inverse_kinematics
        """

        equations = []
        for r in range(3):  # As 3 primeiras linhas são as mais importantes
            for c in range(4):
                equations.append(sp.Eq(Tc[r, c], T_next[r, c]))

        # Identifica as variáveis a serem resolvidas
        unknown_vars = list(T_next.free_symbols)

        if not unknown_vars:
            print("Nenhuma variável simbólica para resolver nesta iteração.")
        else:
            try:
                # Pede ao usuário um "chute inicial" para cada variável
                print("O solucionador numérico precisa de um chute inicial.")
                initial_guesses = []
                for var in unknown_vars:
                    guess_str = input(f"Chute inicial para {var} (padrão é 0): ").strip() or "0"
                    initial_guesses.append(float(guess_str))

                # Prepara os argumentos e chama o nsolve
                # nsolve(equações, variáveis, chutes_iniciais)
                solutions_num = sp.nsolve(equations, unknown_vars, initial_guesses)

                # Exibe a solução numérica encontrada
                print("\nSolução numérica encontrada:")
                for var, val in zip(unknown_vars, solutions_num):
                    val_float = float(val)
                    print(f"  {var} = {val_float:.4f} (rad)  =>  {np.rad2deg(val_float):.2f} (graus)")

            except Exception as e:
                print("\nNão foi possível encontrar uma solução numérica.")
                print("Isso pode acontecer por algumas razões:")
                print(" - O 'chute inicial' está muito longe da solução real.")
                print(" - A posição/orientação desejada é inalcançável pelo robô.")
                print(f"  (Erro do solver: {e})")

if __name__ == "__main__":
    calculate_inverse_kinematic()